## Importing the modules/libraries

In [1]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from geopy.distance import great_circle
import pandas as pd
import numpy as np
import time

In [2]:

df3 = pd.read_csv('A13A-2021-2023-1hour-vessel-movements-report_2022_05_23_merged.csv')
df4 = pd.read_csv('A13A-2021-2023-1hour-vessel-movements-report_2022_08_08_merged.csv')
df5 = pd.read_csv('A13A-2021-2023-1hour-vessel-movements-report_2023_01_02_merged.csv')
df6 = pd.read_csv('A13A-2021-2023-1hour-vessel-movements-report_2023_02_27_merged.csv')
df7 = pd.read_csv('A13A-2021-2023-1hour-vessel-movements-report_2023_03_06_merged.csv')
df8 = pd.read_csv('A13A-2021-2023-1hour-vessel-movements-report_2023_06_12_merged.csv')


# Assuming df1, df2, df3 are the DataFrames to merge
df = pd.concat([ df3, df4, df5, df6, df7, df8], ignore_index=True)

# Reset index after concatenation if needed
df.reset_index(drop=True, inplace=True)

## Dataframe creation & variable formatting

In [2]:
df = pd.read_csv('edge_df_200.csv')

In [6]:
# specifiying path and file format
path = 'C:/Users/golov/Documents/maritine_analysis_home_pc/2021-2023-1hour-vessel-movements-report-weekly/'
extension = '.csv'

# creating a list of csv file names
files = [file for file in os.listdir(path) if file.endswith(extension)]

# importing the files into pandas
dfs = []
for file in files:
    df = pd.read_csv(os.path.join(path, file))
    dfs.append(df)

#concatinating into one dataframe
df = pd.concat(dfs, ignore_index=True)

In [3]:
df.columns

Index(['Time', 'Ship1', 'Ship2', 'latitude_1', 'longitude_1', 'latitude_2',
       'longitude_2', 'Flag1', 'Flag2', 'Owner1', 'Owner2', 'Manager1',
       'Manager2', 'MMSI1', 'MMSI2', 'DRAUGHT1', 'DRAUGHT2', 'SPEED1',
       'SPEED2', 'NAVSTAT1', 'NAVSTAT2', 'COURSE1', 'COURSE2', 'NAME1',
       'NAME2', 'HEADING1', 'HEADING2', 'CALLSIGN1', 'CALLSIGN2', 'AISTYPE1',
       'AISTYPE2', 'DESTINATION1', 'DESTINATION2', 'ETA1', 'ETA2', 'BUILT1',
       'BUILT2', 'TYPENAME1', 'TYPENAME2', 'GT1', 'GT2', 'DWT1', 'DWT2',
       'LOA1', 'LOA2', 'BEAM1', 'BEAM2', 'CLASS1', 'CLASS2', 'CLASSCODE1',
       'CLASSCODE2', 'BUILDER1', 'BUILDER2', 'A1', 'B1', 'C1', 'D1', 'A2',
       'B2', 'C2', 'D2', 'sorted_pair', 'lat_long_date_hour'],
      dtype='object')

In [3]:
# Saving the IMO identifier as a string
df["IMO_str"] = df["IMO"].astype("str")

# Storing the the date time variable in datetime format
df["date_time"]= pd.to_datetime(df["DATE TIME (UTC)"])

# Keeping only year, month, day, and hour (leaving out minutes and seconds)
t_truncated = df["date_time"].dt.floor('h') 

# converting into a string without minutes and seconds 
df["date_time_short_str"] = t_truncated.dt.strftime('%Y-%m-%d %H')

In [4]:
df['flag0'] = None

# updated mmsi prefixes and corresponding country flags
mmsi_flags = {
    201: 'Albania',
    202: 'Andorra',
    203: 'Austria',
    204: 'Portugal',
    205: 'Belgium',
    206: 'Belarus',
    207: 'Bulgaria',
    208: 'Vatican',
    209: 'Cyprus',
    210: 'Cyprus',
    211: 'Germany',
    212: 'Cyprus',
    213: 'Georgia',
    214: 'Moldova',
    215: 'Malta',
    216: 'Armenia',
    218: 'Germany',
    219: 'Denmark',
    220: 'Denmark',
    224: 'Spain',
    225: 'Spain',
    226: 'France',
    227: 'France',
    228: 'France',
    229: 'Malta',
    230: 'Finland',
    231: 'Faroe Islands',
    232: 'United Kingdom',
    233: 'United Kingdom',
    234: 'United Kingdom',
    235: 'United Kingdom',
    236: 'Gibraltar',
    237: 'Greece',
    238: 'Croatia',
    239: 'Greece',
    240: 'Greece',
    241: 'Greece',
    242: 'Morocco',
    243: 'Hungary',
    244: 'Netherlands',
    245: 'Netherlands',
    246: 'Netherlands',
    247: 'Italy',
    248: 'Malta',
    249: 'Malta',
    250: 'Ireland',
    251: 'Iceland',
    252: 'Liechtenstein',
    253: 'Luxembourg',
    254: 'Monaco',
    255: 'Portugal',
    256: 'Malta',
    257: 'Norway',
    258: 'Norway',
    259: 'Norway',
    261: 'Poland',
    262: 'Montenegro',
    263: 'Portugal',
    264: 'Romania',
    265: 'Sweden',
    266: 'Sweden',
    267: 'Slovakia',
    268: 'San Marino',
    269: 'Switzerland',
    270: 'Czech Republic',
    271: 'Turkey',
    272: 'Ukraine',
    273: 'Russia',
    274: 'FYR Macedonia',
    275: 'Latvia',
    276: 'Estonia',
    277: 'Lithuania',
    278: 'Slovenia',
    279: 'Serbia',
    301: 'Anguilla',
    303: 'USA',
    304: 'Antigua and Barbuda',
    305: 'Antigua and Barbuda',
    306: 'Curacao',
    307: 'Aruba',
    308: 'Bahamas',
    309: 'Bahamas',
    310: 'Bermuda',
    311: 'Bahamas',
    312: 'Belize',
    314: 'Barbados',
    316: 'Canada',
    319: 'Cayman Islands',
    321: 'Costa Rica',
    323: 'Cuba',
    325: 'Dominica',
    327: 'Dominican Republic',
    329: 'Guadeloupe',
    330: 'Grenada',
    331: 'Greenland',
    332: 'Guatemala',
    334: 'Honduras',
    336: 'Haiti',
    338: 'USA',
    339: 'Jamaica',
    341: 'Saint Kitts and Nevis',
    343: 'Saint Lucia',
    345: 'Mexico',
    347: 'Martinique',
    348: 'Montserrat',
    350: 'Nicaragua',
    351: 'Panama',
    352: 'Panama',
    353: 'Panama',
    354: 'Panama',
    355: 'Panama',
    356: 'Panama',
    357: 'Panama',
    358: 'Puerto Rico',
    359: 'El Salvador',
    361: 'Saint Pierre and Miquelon',
    362: 'Trinidad and Tobago',
    364: 'Turks and Caicos Islands',
    366: 'USA',
    367: 'USA',
    368: 'USA',
    369: 'USA',
    370: 'Panama',
    371: 'Panama',
    372: 'Panama',
    373: 'Panama',
    374: 'Panama',
    375: 'Saint Vincent and the Grenadines',
    376: 'Saint Vincent and the Grenadines',
    377: 'Saint Vincent and the Grenadines',
    378: 'British Virgin Islands',
    379: 'United States Virgin Islands',
    401: 'Afghanistan',
    403: 'Saudi Arabia',
    405: 'Bangladesh',
    408: 'Bahrain',
    410: 'Bhutan',
    412: 'China',
    413: 'China',
    414: 'China',
    416: 'Taiwan',
    417: 'Sri Lanka',
    419: 'India',
    422: 'Iran',
    423: 'Azerbaijan',
    425: 'Iraq',
    428: 'Israel',
    431: 'Japan',
    432: 'Japan',
    434: 'Turkmenistan',
    436: 'Kazakhstan',
    437: 'Uzbekistan',
    438: 'Jordan',
    440: 'South Korea',
    441: 'South Korea',
    443: 'Palestine',
    445: 'North Korea',
    447: 'Kuwait',
    450: 'Lebanon',
    451: 'Kyrgyz Republic',
    453: 'Macao',
    455: 'Maldives',
    457: 'Mongolia',
    459: 'Nepal',
    461: 'Oman',
    463: 'Pakistan',
    466: 'Qatar',
    468: 'Syria',
    470: 'UAE',
    471: 'UAE',
    472: 'Tajikistan',
    473: 'Yemen',
    475: 'Yemen',
    477: 'Hong Kong',
    478: 'Bosnia and Herzegovina',
    501: 'Antarctica',
    503: 'Australia',
    506: 'Myanmar',
    508: 'Brunei',
    510: 'Micronesia',
    511: 'Palau',
    512: 'New Zealand',
    514: 'Cambodia',
    515: 'Cambodia',
    516: 'Christmas Island',
    518: 'Cook Islands',
    520: 'Fiji',
    523: 'Cocos Islands',
    525: 'Indonesia',
    529: 'Kiribati',
    531: 'Laos',
    533: 'Malaysia',
    536: 'Northern Mariana Islands',
    538: 'Marshall Islands',
    540: 'New Caledonia',
    542: 'Niue',
    544: 'Nauru',
    546: 'French Polynesia',
    548: 'Philippines',
    553: 'Papua New Guinea',
    555: 'Pitcairn Islands',
    557: 'Solomon Islands',
    559: 'American Samoa',
    561: 'Samoa',
    563: 'Singapore',
    564: 'Singapore',
    565: 'Singapore',
    566: 'Singapore',
    567: 'Thailand',
    570: 'Tonga',
    572: 'Tuvalu',
    574: 'Vietnam',
    576: 'Vanuatu',
    577: 'Vanuatu',
    578: 'Wallis and Futuna Islands',
    601: 'South Africa',
    603: 'Angola',
    605: 'Algeria',
    607: 'Saint Paul and Amsterdam Islands',
    608: 'Ascension Island',
    609: 'Burundi',
    610: 'Benin',
    611: 'Botswana',
    612: 'Central African Republic',
    613: 'Cameroon',
    615: 'Congo',
    616: 'Comoros',
    617: 'Cape Verde',
    618: 'Antarctica',
    619: 'Ivory Coast',
    620: 'Comoros',
    621: 'Djibouti',
    622: 'Egypt',
    624: 'Ethiopia',
    625: 'Eritrea',
    626: 'Gabon',
    627: 'Ghana',
    629: 'Gambia',
    630: 'Guinea-Bissau',
    631: 'Equatorial Guinea',
    632: 'Guinea',
    633: 'Burkina Faso',
    634: 'Kenya',
    635: 'Antarctica',
    636: 'Liberia',
    637: 'Liberia',
    642: 'Libya',
    644: 'Lesotho',
    645: 'Mauritius',
    647: 'Madagascar',
    649: 'Mali',
    650: 'Mozambique',
    654: 'Mauritania',
    655: 'Malawi',
    656: 'Niger',
    657: 'Nigeria',
    659: 'Namibia',
    660: 'Reunion',
    661: 'Rwanda',
    662: 'Sudan',
    663: 'Senegal',
    664: 'Seychelles',
    665: 'Saint Helena',
    666: 'Somalia',
    667: 'Sierra Leone',
    668: 'Sao Tome and Principe',
    669: 'Eswatini',
    670: 'Chad',
    671: 'Togo',
    672: 'Tunisia',
    674: 'Tanzania',
    675: 'Uganda',
    676: 'DR Congo',
    677: 'Tanzania',
    678: 'Zambia',
    679: 'Zimbabwe',
    701: 'Argentina',
    710: 'Brazil',
    720: 'Bolivia',
    725: 'Chile',
    730: 'Colombia',
    735: 'Ecuador',
    740: 'United Kingdom',
    745: 'Guiana',
    750: 'Guyana',
    755: 'Paraguay',
    760: 'Peru',
    765: 'Suriname',
    770: 'Uruguay',
    775: 'Venezuela'
}

# assign flags based on the mmsi prefix
for i, mmsi in enumerate(df['MMSI']):
    prefix = int(str(mmsi)[:3])  
    df.at[i, 'flag0'] = mmsi_flags.get(prefix, 'Unknown Flag')


## Defining the functions for Ship-to-ship detections

In [5]:
def haversine_vectorized(lat1, lon1, lat2, lon2): #optimized for long arrays #calculate shortest distance between two points (points being the boat) Is th point defined by the antena?
    R = 6371000  # Earth radius in meters
    phi1 = np.radians(lat1) #radians to convert from degrees to radians
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

#it would be ere the distance calculator where it saves the values in latitude and longitude variables? or better distance variables which are operationalized as lat, long all we have to do is substitute them in the next function 

#this function gets the boats that are next to eachother at a specific distance and registeres on the hour when this occurs 
def create_edgelist(input_df, max_distance):
    # Initialize an empty list for edges
    edges = []

#     
    #     'DESTINATION', 'ETA', 'BUILT', 'flag0',
    #    'TYPENAME', 'GT', 'DWT', 'LOA', 'BEAM', 'DRAUGHT.1', 'CLASS',
    #    'CLASSCODE', 'BUILDER', 'MANAGER_NAME', 'OWNER_NAME', 'IMO_str',
    #    'date_time', 'date_time_short_str']

    # Group by rounded down hour and iterate through each group
    for time, group in input_df.groupby('date_time_short_str'): #group by hour 
        ships = group['IMO_str'].tolist()  # Ship names (IMO numbers)
        latitudes = group['LATITUDE'].values  # Latitude
        longitudes = group['LONGITUDE'].values  # Longitude
        flags = group['flag0'].tolist()  # Retrieve flag
        owners = group['OWNER_NAME'].tolist()  # Retrieve owner
        managers = group['MANAGER_NAME'].tolist()  # Retrieve manager
        mmsi = group['MMSI'].tolist()  # Retrieve MMSI for each ship (Change 1)
        draught = group['DRAUGHT'].tolist() # Retrieve DRAUGHT for each ship
        speed = group['SPEED'].tolist() # SPEED (SOG)
        navstat = group['NAVSTAT'].tolist() #NAVSTAT
        course = group['COURSE'].tolist() #COURSE
        name = group['NAME'].tolist() #NAME
        heading = group['HEADING'].tolist() #HEADING
        callsign = group['CALLSIGN'].tolist() #CALLSIGN
        aistype = group['AISTYPE'].tolist() #AISTYPE
        a = group['A'].tolist()
        b = group['B'].tolist()
        c = group['C'].tolist()
        d = group['D'].tolist()
        destination = group['DESTINATION'].tolist()  # Retrieve DESTINATION
        eta = group['ETA'].tolist()  # Retrieve ETA
        built = group['BUILT'].tolist()  # Retrieve BUILT
        typename = group['TYPENAME'].tolist()  # Retrieve ship type
        gt = group['GT'].tolist()  # Gross Tonnage
        dwt = group['DWT'].tolist()  # Deadweight Tonnage
        loa = group['LOA'].tolist()  # Length Overall
        beam = group['BEAM'].tolist()  # Beam (width)
        ship_class = group['CLASS'].tolist()  # Class of ship
        class_code = group['CLASSCODE'].tolist()  # Ship class code
        builder = group['BUILDER'].tolist()  # Shipbuilder name
     

        
        # Use NumPy to create a meshgrid for latitudes and longitudes
        lat1, lat2 = np.meshgrid(latitudes, latitudes)
        lon1, lon2 = np.meshgrid(longitudes, longitudes)

        # Calculate distances using the vectorized Haversine function
        distances = haversine_vectorized(lat1, lon1, lat2, lon2)

        # Create a mask for pairs within max_distance and not the same ship
        mask = (np.round(distances, 0) <= max_distance) & (np.arange(len(ships))[:, None] != np.arange(len(ships)))


        # Get ship pairs that satisfy the distance condition
        for i, j in zip(*np.where(mask)):
            edges.append((
                time, ships[i], ships[j], latitudes[i], longitudes[i], latitudes[j], longitudes[j],
                flags[i], flags[j], owners[i], owners[j], managers[i], managers[j], mmsi[i], mmsi[j],
                draught[i], draught[j], speed[i], speed[j], navstat[i], navstat[j], course[i], course[j],
                name[i], name[j], heading[i], heading[j], callsign[i], callsign[j], aistype[i], aistype[j],
                destination[i], destination[j], eta[i], eta[j], built[i], built[j], typename[i], typename[j],
                gt[i], gt[j], dwt[i], dwt[j], loa[i], loa[j], beam[i], beam[j],
                ship_class[i], ship_class[j], class_code[i], class_code[j], builder[i], builder[j], 
                a[i], b[i], c[i], d[i], a[j], b[j], c[j], d[j]
            ))
                
        # Update edges list with additional attributes


    # Create a new DataFrame for the edge list with additional columns
    edge_df = pd.DataFrame(edges, columns=[
        'Time', 'Ship1', 'Ship2', 'latitude_1', 'longitude_1', 'latitude_2', 'longitude_2',
        'Flag1', 'Flag2', 'Owner1', 'Owner2', 'Manager1', 'Manager2', 'MMSI1', 'MMSI2',
        'DRAUGHT1', 'DRAUGHT2', 'SPEED1', 'SPEED2', 'NAVSTAT1', 'NAVSTAT2', 'COURSE1', 'COURSE2',
        'NAME1', 'NAME2', 'HEADING1', 'HEADING2', 'CALLSIGN1', 'CALLSIGN2', 'AISTYPE1', 'AISTYPE2',
        'DESTINATION1', 'DESTINATION2', 'ETA1', 'ETA2', 'BUILT1', 'BUILT2', 'TYPENAME1', 'TYPENAME2',
        'GT1', 'GT2', 'DWT1', 'DWT2', 'LOA1', 'LOA2', 'BEAM1', 'BEAM2', 
        'CLASS1', 'CLASS2', 'CLASSCODE1', 'CLASSCODE2', 'BUILDER1', 'BUILDER2',
        'A1','B1','C1', 'D1', 'A2','B2', 'C2', 'D2'    ])



    

    edge_df['sorted_pair'] = edge_df.apply(
        lambda row: tuple(sorted([row['Ship1'], row['Ship2']])), axis=1
    )
    edge_df = edge_df.sort_values(by=['Time', 'sorted_pair', 'Ship1'], ascending=[True, True, False])
																			# check the time because im not sure that they apply. 
    edge_df_cleaned = edge_df.drop_duplicates(subset=['Time', 'sorted_pair'], keep='first')
    

  

    
    # edge_df_cleaned = edge_df.iloc[::2].reset_index(drop=True) #takes out duplicates #this does not work
    
    # Creating a string that combines all of the information and removing duplicates
    edge_df_cleaned["lat_long_date_hour"] = (
        edge_df_cleaned["latitude_1"].astype('str') + ' ' +
        edge_df_cleaned["longitude_1"].astype('str') + ' ' +
        edge_df_cleaned["Time"]
    )
    
    edge_df_cleaned = edge_df_cleaned.drop_duplicates(subset=['lat_long_date_hour', 'Ship1', 'Ship2']) 
    
    # Remove self-ties (i.e., if source and target nodes are the same)
    edge_df_cleaned = edge_df_cleaned[edge_df_cleaned['Ship1'] != edge_df_cleaned['Ship2']].reset_index(drop=True)

    return edge_df_cleaned

Function that only keeps edges if the ships have been close to each other for at least n consequtive hours:

In [37]:
def find_consecutive_adjacency(edgelist_df, min_hours, gap_tolerance_hours=2):
    # Ensure the Time column is in datetime format
    edgelist_df['Time'] = pd.to_datetime(edgelist_df['Time'])

    # Initialize a list to store new edges
    new_edges = []

    # Group by Ship1 and Ship2
    grouped = edgelist_df.groupby(['Ship1', 'Ship2'])

    for (ship1, ship2), group in grouped:
        # Sort by Time
        group = group.sort_values('Time')
        # Reset index for easier processing
        group = group.reset_index(drop=True)

        # Check for consecutive hours
        count = 0 # because we add the time threshold gap of one hour  
        # Start Time
        start_time = group['Time'].iloc[0]
        #Initial draught
        initial_draught_1 = group['DRAUGHT1'].iloc[0]
        initial_draught_2 = group['DRAUGHT2'].iloc[0]
        #Initial Location
        initial_latitude_1 = group['latitude_1'].iloc[0]
        initial_longitude_1 = group['longitude_1'].iloc[0]
        initial_latitude_2 = group['latitude_2'].iloc[0]
        initial_longitude_2 = group['longitude_2'].iloc[0]
        #Initial Course
        initial_course_1 = group['COURSE1'].iloc[0]
        initial_course_2 = group['COURSE2'].iloc[0]
        # initial heading 
        initial_heading_1 = group['HEADING1'].iloc[0]
        initial_heading_2 = group['HEADING2'].iloc[0]
        # initial name 
        initial_name_1 = group['NAME1'].iloc[0]
        initial_name_2 = group['NAME2'].iloc[0]
        #initial destination 
        initial_destination_1 = group['DESTINATION1'].iloc[0]
        initial_destination_2 = group['DESTINATION2'].iloc[0]
        # initial eta
        initial_eta_1 = group['ETA1'].iloc[0]
        initial_eta_2 = group['ETA2'].iloc[0]
        # initial flag 
        initial_flag_1 = group['Flag1'].iloc[0]
        initial_flag_2 = group['Flag2'].iloc[0]
        #MMSI 
        initial_mmsi1 = group['MMSI1'].iloc[0]
        initial_mmsi2 = group['MMSI2'].iloc[0]
        # initial manager 
        initial_manager_1 = group['Manager1'].iloc[0]
        initial_manager_2 = group['Manager2'].iloc[0]
        # initial owner 
        initial_owner_1 = group['Owner1'].iloc[0]
        initial_owner_2 = group['Owner2'].iloc[0]

        for i in range(1, len(group)): 
            time_diff = (group['Time'].iloc[i] - group['Time'].iloc[i - 1]).total_seconds() / 3600

            if time_diff == 1:  # 1-hour difference
                count += 1
            if time_diff <= gap_tolerance_hours:  
                count += 1
            else:  # If gap is larger than tolerance, finalize the current session
                if count >= min_hours:
                    
                    # Variables that can change through time: 

                    # End Time
                    end_time = group['Time'].iloc[i - 1]

                    # Final Location
                    final_latitude_1 = group['latitude_1'].iloc[i - 1]  # Final latitude for the session
                    final_longitude_1 = group['longitude_1'].iloc[i - 1]  # Final longitude for the session
                    final_latitude_2 = group['latitude_2'].iloc[i - 1]  # Final latitude for the session
                    final_longitude_2 = group['longitude_2'].iloc[i - 1]  # Final longitude for the session

                    # Final Draught
                    final_draught_1 = group['DRAUGHT1'].iloc[i - 1]
                    final_draught_2 = group['DRAUGHT2'].iloc[i - 1]

                    # Final Course
                    final_course_1 = group['COURSE1'].iloc[i - 1]
                    final_course_2 = group['COURSE2'].iloc[i - 1]

                    # Final Heading
                    final_heading_1 = group['HEADING1'].iloc[i - 1]
                    final_heading_2 = group['HEADING2'].iloc[i - 1]

                    # Final Name
                    final_name_1 = group['NAME1'].iloc[i - 1]
                    final_name_2 = group['NAME2'].iloc[i - 1]

                    # Final Destination
                    final_destination_1 = group['DESTINATION1'].iloc[i - 1]
                    final_destination_2 = group['DESTINATION2'].iloc[i - 1]

                    # Final ETA
                    final_eta_1 = group['ETA1'].iloc[i - 1]
                    final_eta_2 = group['ETA2'].iloc[i - 1]

                    # Final Flag
                    final_flag_1 = group['Flag1'].iloc[i - 1]
                    final_flag_2 = group['Flag2'].iloc[i - 1]

                    # mmsi
                    final_mmsi1 = group['MMSI1'].iloc[i-1]
                    final_mmsi2 = group['MMSI2'].iloc[i-1]

                    # Final Manager
                    final_manager_1 = group['Manager1'].iloc[i - 1]
                    final_manager_2 = group['Manager2'].iloc[i - 1]

                    # Final Owner
                    final_owner_1 = group['Owner1'].iloc[i - 1]
                    final_owner_2 = group['Owner2'].iloc[i - 1]



           

                    new_edges.append((
                        ship1, 
                        ship2, 
                        start_time, 
                        end_time, 
                        (end_time - start_time).total_seconds() / 3600,
                        group['mean_speed'].iloc[0],
                        initial_flag_1,  # FLAG_NAME for Ship1
                        final_flag_1,
                        initial_flag_2, 
                        final_flag_2,
                        initial_mmsi1, 
                        final_mmsi1,
                        initial_mmsi2, 
                        final_mmsi2,
                        initial_owner_1,
                        final_owner_1,
                        initial_owner_2,
                        final_owner_2,
                        initial_manager_1,
                        final_manager_1,
                        initial_manager_2,
                        final_manager_2,
                        # group['MMSI1'].iloc[0],  # MMSI for Ship1
                        initial_draught_1, #DRAUGHT for Ship1
                        final_draught_1,
                        # group['MMSI2'].iloc[0],   # MMSI for Ship2
                        initial_draught_2, #DRAUGHT for Ship2
                        final_draught_2,
                        initial_latitude_1,         # Initial latitude
                        initial_longitude_1,        # Initial longitude
                        final_latitude_1,           # Final latitude
                        final_longitude_1,           # Final longitude
                        initial_latitude_2,         # Initial latitude
                        initial_longitude_2,        # Initial longitude
                        final_latitude_2,           # Final latitude
                        final_longitude_2,           # Final longitude
                        initial_course_1,
                        final_course_1,
                        initial_course_2,
                        final_course_2,
                        group['TYPENAME1'].iloc[0],
                        group['TYPENAME2'].iloc[0], 
                        group['BUILT1'].iloc[0],
                        group['BUILT2'].iloc[0],
                        group['GT1'].iloc[0],
                        group['GT2'].iloc[0],
                        group['DWT1'].iloc[0],
                        group['DWT2'].iloc[0],
                        group['LOA1'].iloc[0],
                        group['LOA2'].iloc[0],
                        group['BEAM1'].iloc[0],
                        group['BEAM2'].iloc[0],
                        group['CLASS1'].iloc[0],
                        group['CLASS2'].iloc[0],
                        group['CLASSCODE1'].iloc[0],
                        group['CLASSCODE2'].iloc[0],
                        group['BUILDER1'].iloc[0],
                        group['BUILDER2'].iloc[0],
                        group['mean_A1'].iloc[0],
                        group['mean_B1'].iloc[0],
                        group['mean_C1'].iloc[0],
                        group['mean_D1'].iloc[0],
                        group['mean_A2'].iloc[0],
                        group['mean_B2'].iloc[0],
                        group['mean_C2'].iloc[0],
                        group['mean_D2'].iloc[0],
                        group['median_navstat1'].iloc[0],
                        group['median_navstat2'].iloc[0],
                        initial_heading_1,
                        final_heading_1,
                        initial_heading_2,
                        final_heading_2,
                        initial_name_1,
                        final_name_1,
                        initial_name_2,
                        final_name_2, 
                        initial_destination_1, 
                        final_destination_1,
                        initial_destination_2, 
                        final_destination_2,
                        initial_eta_1,
                        final_eta_1,
                        initial_eta_2,
                        final_eta_2
                        

                    ))  # Save Start and End time, and the difference
                count = 0  # Reset count

                # variables that can change with time
                start_time = group['Time'].iloc[i]  # Update start time


                
                
                initial_latitude_1 = group['latitude_1'].iloc[i]  # Update initial latitude for the new session
                initial_longitude_1 = group['longitude_1'].iloc[i]  # Update initial longitude for the new session
                initial_latitude_2 = group['latitude_2'].iloc[i]  # Update initial latitude for the new session
                initial_longitude_2 = group['longitude_2'].iloc[i]  # Update initial longitude for the new session
                initial_draught_1 = group['DRAUGHT1'].iloc[i]
                initial_draught_2 = group['DRAUGHT2'].iloc[i]
                initial_course_1 = group['COURSE1'].iloc[i]
                initial_course_2 = group['COURSE2'].iloc[i]
                # initial heading 
                initial_heading_1 = group['HEADING1'].iloc[i]
                initial_heading_2 = group['HEADING2'].iloc[i]
                # initial name 
                initial_name_1 = group['NAME1'].iloc[i]
                initial_name_2 = group['NAME2'].iloc[i]
                #initial destination 
                initial_destination_1 = group['DESTINATION1'].iloc[i]
                initial_destination_2 = group['DESTINATION2'].iloc[i]
                # initial eta
                initial_eta_1 = group['ETA1'].iloc[i]
                initial_eta_2 = group['ETA2'].iloc[i]
                # initial flag 
                initial_flag_1 = group['Flag1'].iloc[i]
                initial_flag_2 = group['Flag2'].iloc[i]
                #mmsi
                initial_mmsi1 = group['MMSI1'].iloc[i]
                initial_mmsi2 = group['MMSI2'].iloc[i]
                # initial manager 
                initial_manager_1 = group['Manager1'].iloc[i]
                initial_manager_2 = group['Manager2'].iloc[i]
                # initial owner 
                initial_owner_1 = group['Owner1'].iloc[i]
                initial_owner_2 = group['Owner2'].iloc[i]

        # Check for the last session
        if count >= min_hours: 
        # if count >= min_hours or (len(group) > 1 and (group['Time'].iloc[len(group) - 1] - group['Time'].iloc[len(group) - 2]).total_seconds() <= 3600):
            end_time = group['Time'].iloc[len(group) - 1]

            final_latitude_1 = group['latitude_1'].iloc[len(group) - 1]  # Final latitude for the last session
            final_longitude_1 = group['longitude_1'].iloc[len(group) - 1]  # Final longitude for the last session
            final_latitude_2 = group['latitude_2'].iloc[len(group) - 1]  # Final latitude for the last session
            final_longitude_2 = group['longitude_2'].iloc[len(group) - 1]  # Final longitude for the last session
            
            final_draught_1 = group['DRAUGHT1'].iloc[len(group) - 1]
            final_draught_2 = group['DRAUGHT2'].iloc[len(group) - 1]
            final_course_1 = group['COURSE1'].iloc[len(group) - 1]
            final_course_2 = group['COURSE2'].iloc[len(group) - 1]

            # Final Heading
            final_heading_1 = group['HEADING1'].iloc[len(group) - 1]
            final_heading_2 = group['HEADING2'].iloc[len(group) - 1]

            # Final Name
            final_name_1 = group['NAME1'].iloc[len(group) - 1]
            final_name_2 = group['NAME2'].iloc[len(group) - 1]

            # Final Destination
            final_destination_1 = group['DESTINATION1'].iloc[len(group) - 1]
            final_destination_2 = group['DESTINATION2'].iloc[len(group) - 1]

            # Final ETA
            final_eta_1 = group['ETA1'].iloc[len(group) - 1]
            final_eta_2 = group['ETA2'].iloc[len(group) - 1]

            # Final Flag
            final_flag_1 = group['Flag1'].iloc[len(group) - 1]
            final_flag_2 = group['Flag2'].iloc[len(group) - 1]

            # mmsi
            final_mmsi1 = group['MMSI1'].iloc[len(group) - 1]
            final_mmsi2 = group['MMSI2'].iloc[len(group) - 1]

            # Final Manager
            final_manager_1 = group['Manager1'].iloc[len(group) - 1]
            final_manager_2 = group['Manager2'].iloc[len(group) - 1]

            # Final Owner
            final_owner_1 = group['Owner1'].iloc[len(group) - 1]
            final_owner_2 = group['Owner2'].iloc[len(group) - 1]


            new_edges.append((
                ship1, 
                ship2, 
                start_time, 
                end_time, 
                (end_time - start_time).total_seconds() / 3600,
                group['mean_speed'].iloc[0],
                initial_flag_1,  # FLAG_NAME for Ship1
                final_flag_1,
                initial_flag_2, 
                final_flag_2,
                initial_mmsi1, 
                final_mmsi1,
                initial_mmsi2, 
                final_mmsi2,
                initial_owner_1,
                final_owner_1,
                initial_owner_2,
                final_owner_2,
                initial_manager_1,
                final_manager_1,
                initial_manager_2,
                final_manager_2,
                # group['MMSI1'].iloc[0],  # MMSI for Ship1
                initial_draught_1, #DRAUGHT for Ship1
                final_draught_1,
                # group['MMSI2'].iloc[0],   # MMSI for Ship2
                initial_draught_2, #DRAUGHT for Ship2
                final_draught_2,
                initial_latitude_1,         # Initial latitude
                initial_longitude_1,        # Initial longitude
                final_latitude_1,           # Final latitude
                final_longitude_1,           # Final longitude
                initial_latitude_2,         # Initial latitude
                initial_longitude_2,        # Initial longitude
                final_latitude_2,           # Final latitude
                final_longitude_2,           # Final longitude
                initial_course_1,
                final_course_1,
                initial_course_2,
                final_course_2,
                group['TYPENAME1'].iloc[0],
                group['TYPENAME2'].iloc[0], 
                group['BUILT1'].iloc[0],
                group['BUILT2'].iloc[0],
                group['GT1'].iloc[0],
                group['GT2'].iloc[0],
                group['DWT1'].iloc[0],
                group['DWT2'].iloc[0],
                group['LOA1'].iloc[0],
                group['LOA2'].iloc[0],
                group['BEAM1'].iloc[0],
                group['BEAM2'].iloc[0],
                group['CLASS1'].iloc[0],
                group['CLASS2'].iloc[0],
                group['CLASSCODE1'].iloc[0],
                group['CLASSCODE2'].iloc[0],
                group['BUILDER1'].iloc[0],
                group['BUILDER2'].iloc[0],
                group['mean_A1'].iloc[0],
                group['mean_B1'].iloc[0],
                group['mean_C1'].iloc[0],
                group['mean_D1'].iloc[0],
                group['mean_A2'].iloc[0],
                group['mean_B2'].iloc[0],
                group['mean_C2'].iloc[0],
                group['mean_D2'].iloc[0],
                group['median_navstat1'].iloc[0],
                group['median_navstat2'].iloc[0],
                initial_heading_1,
                final_heading_1,
                initial_heading_2,
                final_heading_2,
                initial_name_1,
                final_name_1,
                initial_name_2,
                final_name_2, 
                initial_destination_1, 
                final_destination_1,
                initial_destination_2, 
                final_destination_2,
                initial_eta_1,
                final_eta_1,
                initial_eta_2,
                final_eta_2
                

            ))  # Save Start and End time, and the difference

    # Create a new DataFrame for the new edgelist
    new_edge_df = pd.DataFrame(new_edges, columns=[
        'Ship1', 'Ship2', 'Start time', 'End time', 
        'Consecutive (hours)', 'mean_speed',
        'initial_flag1','final_flag1', 'initial_flag2', 'final_flag2', 'initial_mmsi1', 'final_mmsi1', 'initial_mmsi2', 'final_mmsi2',
        'initial_owner1', 'final_owner1','initial_owner2', 'final_owner2',
        'initial_manager1','final_manager1', 'initial_manager2', 'final_manager2', 
        'initial_draught1', 'final_draught1',
        'initial_draught2', 'final_draught2',
        'initial_latitude1', 'initial_longitude1', 'final_latitude1', 'final_longitude1',
        'initial_latitude2', 'initial_longitude2', 'final_latitude2', 'final_longitude2', 
        'initial_course1', 'final_course1', 'initial_course2', 'final_course2', 
        'TYPENAME1', 'TYPENAME2',
        'BUILT1', 'BUILT2', 'GT1', 'GT2', 'DWT1', 'DWT2', 'LOA1', 'LOA2', 'BEAM1', 'BEAM2', 'CLASS2','CLASS2',
        'CLASSCODE1', 'CLASSCODE2', 'BUILDER1', 'BUILDER2', 
        'mean_A1', 'mean_B1', 'mean_C1', 'mean_D1', 'mean_A2', 'mean_B2', 'mean_C2', 'mean_D2',
        'median_navstat1', 'median_navstat2',
        'initial_heading1', 'final_heading1', 'initial_heading2', 'final_heading2', 
        'initial_name1', 'final_name1', 'initial_name2', 'final_name2',
        'initial_destination1', 'final_destination1', 'initial_destination2', 'final_destination2',
        'initial_eta1', 'final_eta1', 'initial_eta2', 'final_eta2'


        
    ])
    
    new_edge_df['Consecutive (hours)'] = new_edge_df['Consecutive (hours)'] + 1  # Adjusting for counting the overlapping hour sequences
 
    return new_edge_df

Function that merges two edges if the time gap between them is no greater than 24 hours:

In [13]:
def find_consecutive_adjacency_broad(adj_df):
    # Ensure 'Start time' and 'End time' columns are in datetime format
    adj_df['Start time'] = pd.to_datetime(adj_df['Start time'])
    adj_df['End time'] = pd.to_datetime(adj_df['End time'])
    
    # Sort by Ship1, Ship2, and Start time to organize edges chronologically for each ship pair
    adj_df = adj_df.sort_values(['Ship1', 'Ship2', 'Start time']).reset_index(drop=True)
    
    # List to store merged edges
    merged_edges = []

    # Iterate through each unique ship pair
    for (ship1, ship2), group in adj_df.groupby(['Ship1', 'Ship2']):
        # Initialize the first edge as the starting reference for merging
        # Extract the first row values for the group
        start_time = group.iloc[0]['Start time']
        end_time = group.iloc[0]['End time']
        consecutive_hours = group.iloc[0]['Consecutive (hours)']

        # Initialize additional information for merged edge
        initial_flag1 = group.iloc[0]['initial_flag1']
        final_flag1 = group.iloc[0]['final_flag1']
        initial_flag2 = group.iloc[0]['initial_flag2']
        final_flag2 = group.iloc[0]['final_flag2']
        initial_mmsi1 = group.iloc[0]['initial_mmsi1'] 
        final_mmsi1 = group.iloc[0]['final_mmsi1'] 
        initial_mmsi2 = group.iloc[0]['initial_mmsi2']
        final_mmsi2 = group.iloc[0]['final_mmsi2'] 
        initial_owner1 = group.iloc[0]['initial_owner1']
        final_owner1 = group.iloc[0]['final_owner1']
        initial_owner2 = group.iloc[0]['initial_owner2']
        final_owner2 = group.iloc[0]['final_owner2']
        initial_manager1 = group.iloc[0]['initial_manager1']
        final_manager1 = group.iloc[0]['final_manager1']
        initial_manager2 = group.iloc[0]['initial_manager2']
        final_manager2 = group.iloc[0]['final_manager2']
        mean_speed = group.iloc[0]['mean_speed']

        # Initialize draught values
        initial_draught_1 = group.iloc[0]['initial_draught1']
        final_draught_1 = group.iloc[0]['final_draught1']
        initial_draught_2 = group.iloc[0]['initial_draught2']
        final_draught_2 = group.iloc[0]['final_draught2']

        # Calculate absolute draught changes
        absolute_draught_change1 = abs(final_draught_1 - initial_draught_1)
        absolute_draught_change2 = abs(final_draught_2 - initial_draught_2)

        initial_latitude_1 = group.iloc[0]['initial_latitude1']
        initial_longitude_1 = group.iloc[0]['initial_longitude1']
        final_latitude_1 = group.iloc[0]['final_latitude1']
        final_longitude_1 = group.iloc[0]['final_longitude1']
        initial_latitude_2 = group.iloc[0]['initial_latitude2']
        initial_longitude_2 = group.iloc[0]['initial_longitude2']
        final_latitude_2 = group.iloc[0]['final_latitude2']
        final_longitude_2 = group.iloc[0]['final_longitude2']

        initial_course_1 = group.iloc[0]['initial_course1']
        final_course_1 = group.iloc[0]['final_course1']
        initial_course_2 = group.iloc[0]['initial_course2']
        final_course_2 = group.iloc[0]['final_course2']

        typename1 = group.iloc[0]['TYPENAME1']
        typename2 = group.iloc[0]['TYPENAME2']
        built1 = group.iloc[0]['BUILT1']
        built2 = group.iloc[0]['BUILT2']
        gt1 = group.iloc[0]['GT1']
        gt2 = group.iloc[0]['GT2']
        dwt1 = group.iloc[0]['DWT1']
        dwt2 = group.iloc[0]['DWT2']
        loa1 = group.iloc[0]['LOA1']
        loa2 = group.iloc[0]['LOA2']
        beam1 = group.iloc[0]['BEAM1']
        beam2 = group.iloc[0]['BEAM2']

        # Mean speed columns
        mean_A1 = group.iloc[0]['mean_A1']
        mean_B1 = group.iloc[0]['mean_B1']
        mean_C1 = group.iloc[0]['mean_C1']
        mean_D1 = group.iloc[0]['mean_D1']
        mean_A2 = group.iloc[0]['mean_A2']
        mean_B2 = group.iloc[0]['mean_B2']
        mean_C2 = group.iloc[0]['mean_C2']
        mean_D2 = group.iloc[0]['mean_D2']

        # Iterate over edges for this ship pair
        for i in range(1, len(group)):
            next_start = group.iloc[i]['Start time']
            next_end = group.iloc[i]['End time']
            next_final_draught_1 = group.iloc[i]['final_draught1']
            next_final_draught_2 = group.iloc[i]['final_draught2']

            # Check if the gap between the current 'end_time' and 'next_start' is within 24 hours
            if (next_start - end_time).total_seconds() <= 24 * 3600:  # 24 hours in seconds
                # Update the end_time and final draughts to the latest values
                end_time = max(end_time, next_end)
                final_draught_1 = next_final_draught_1
                final_draught_2 = next_final_draught_2
                # dynamically update flags 
                final_flag1 = group.iloc[i]['final_flag1']
                final_flag2 = group.iloc[i]['final_flag2']
            else:
                # Calculate the total hours for the current merged edge
                consecutive_hours = (end_time - start_time).total_seconds() / 3600

                # Append merged edge to the list
                merged_edges.append((
                    ship1, ship2, start_time, end_time, consecutive_hours, mean_speed,
                    initial_latitude_1, initial_longitude_1, final_latitude_1, final_longitude_1, 
                    initial_latitude_2, initial_longitude_2, final_latitude_2, final_longitude_2, 
                    initial_flag1, final_flag1, initial_flag2, final_flag2, initial_mmsi1, final_mmsi1, initial_mmsi2, final_mmsi2,
                    initial_owner1, final_owner1, initial_owner2, final_owner2, 
                    initial_manager1, final_manager1, initial_manager2, final_manager2, 

                    initial_draught_1, final_draught_1, initial_draught_2, final_draught_2,
                    absolute_draught_change1, absolute_draught_change2,
                    mean_A1, mean_B1, mean_C1, mean_D1, mean_A2, mean_B2, mean_C2, mean_D2
                ))

                # Reset for the next segment
                start_time = next_start
                end_time = next_end
                initial_draught_1 = group.iloc[i]['initial_draught1']
                final_draught_1 = next_final_draught_1
                initial_draught_2 = group.iloc[i]['initial_draught2']
                final_draught_2 = next_final_draught_2
                # flags and mmsi
                initial_flag1 = group.iloc[i]['initial_flag1']
                final_flag1 = group.iloc[i]['final_flag1']
                initial_flag2 = group.iloc[i]['initial_flag2']
                final_flag2 = group.iloc[i]['final_flag2']
                initial_mmsi1 = group.iloc[i]['initial_mmsi1']
                final_mmsi1 = group.iloc[i]['final_mmsi1']
                initial_mmsi2 = group.iloc[i]['initial_mmsi2']
                final_mmsi2 = group.iloc[i]['final_mmsi2']

        # Add the last segment for the group
        consecutive_hours = (end_time - start_time).total_seconds() / 3600
        merged_edges.append((
            ship1, ship2, start_time, end_time, consecutive_hours, mean_speed,
            initial_latitude_1, initial_longitude_1, final_latitude_1, final_longitude_1, 
            initial_latitude_2, initial_longitude_2, final_latitude_2, final_longitude_2, 
            initial_flag1, final_flag1, initial_flag2, final_flag2, initial_mmsi1, final_mmsi1, initial_mmsi2, final_mmsi2,
            initial_owner1, final_owner1, initial_owner2, final_owner2, 
            initial_manager1, final_manager1, initial_manager2, final_manager2,  
            initial_draught_1, final_draught_1, initial_draught_2, final_draught_2,
            absolute_draught_change1, absolute_draught_change2,
            mean_A1, mean_B1, mean_C1, mean_D1, mean_A2, mean_B2, mean_C2, mean_D2
        ))

    # Create a DataFrame for the merged edges
    merged_edge_df = pd.DataFrame(merged_edges, columns=[
        'Ship1', 'Ship2', 'Start time', 'End time', 'Consecutive (hours)', 'mean_speed',
        'initial_latitude_1', 'initial_longitude_1', 'final_latitude_1', 'final_longitude_1',
        'initial_latitude_2', 'initial_longitude_2', 'final_latitude_2', 'final_longitude_2',
        'initial_flag1', 'final_flag1', 'initial_flag2', 'final_flag2', 'initial_mmsi1', 'final_mmsi1', 'initial_mmsi2', 'final_mmsi2',
        'initial_owner1', 'final_owner1', 'initial_owner2', 'final_owner2',
        'initial_manager1', 'final_manager1', 'initial_manager2', 'final_manager2', 
        'initial_draught_1', 'final_draught_1', 
        'initial_draught_2', 'final_draught_2',
        'absolute_draught_change1', 'absolute_draught_change2',
        'mean_A1', 'mean_B1', 'mean_C1', 'mean_D1', 'mean_A2', 'mean_B2', 'mean_C2', 'mean_D2'
    ])
    
    return merged_edge_df


Function that calculates the mean:

In [14]:
def mean_values(group):
    # Ensure the group is sorted by time
    group = group.sort_values('Time').reset_index(drop=True)
    
    # Initialize placeholders for mean values
    placeholders = ['mean_speed', 'median_navstat1', 'median_navstat2', 
                    'mean_A1', 'mean_B1', 'mean_C1', 'mean_D1',
                    'mean_A2', 'mean_B2', 'mean_C2', 'mean_D2']
    for placeholder in placeholders:
        group[placeholder] = np.nan

    # Start processing in 720-hour chunks
    chunk_start_idx = 0
    while chunk_start_idx < len(group):
        start_time = group.loc[chunk_start_idx, 'Time']
        end_time = start_time + pd.Timedelta(hours=720)
        chunk = group[(group['Time'] >= start_time) & (group['Time'] < end_time)]

        if not chunk.empty and len(chunk) > 1:  # Ensure data exists for processing
            # Perform calculations
            mean_speed1 = chunk['SPEED1'].mean() if 'SPEED1' in chunk else np.nan
            mean_speed2 = chunk['SPEED2'].mean() if 'SPEED2' in chunk else np.nan
            overall_mean_speed = np.nanmean([mean_speed1, mean_speed2])

            median_navstat1 = chunk['NAVSTAT1'].median() if 'NAVSTAT1' in chunk else np.nan
            median_navstat2 = chunk['NAVSTAT2'].median() if 'NAVSTAT2' in chunk else np.nan

            # Calculate mean A, B, C, D values if columns exist
            mean_A1 = chunk['A1'].mean() if 'A1' in chunk else np.nan
            mean_B1 = chunk['B1'].mean() if 'B1' in chunk else np.nan
            mean_C1 = chunk['C1'].mean() if 'C1' in chunk else np.nan
            mean_D1 = chunk['D1'].mean() if 'D1' in chunk else np.nan
            mean_A2 = chunk['A2'].mean() if 'A2' in chunk else np.nan
            mean_B2 = chunk['B2'].mean() if 'B2' in chunk else np.nan
            mean_C2 = chunk['C2'].mean() if 'C2' in chunk else np.nan
            mean_D2 = chunk['D2'].mean() if 'D2' in chunk else np.nan

            # Assign calculated values to the chunk
            group.loc[chunk.index, 'mean_speed'] = overall_mean_speed
            group.loc[chunk.index, 'median_navstat1'] = median_navstat1
            group.loc[chunk.index, 'median_navstat2'] = median_navstat2
            group.loc[chunk.index, 'mean_A1'] = mean_A1
            group.loc[chunk.index, 'mean_B1'] = mean_B1
            group.loc[chunk.index, 'mean_C1'] = mean_C1
            group.loc[chunk.index, 'mean_D1'] = mean_D1
            group.loc[chunk.index, 'mean_A2'] = mean_A2
            group.loc[chunk.index, 'mean_B2'] = mean_B2
            group.loc[chunk.index, 'mean_C2'] = mean_C2
            group.loc[chunk.index, 'mean_D2'] = mean_D2

        chunk_start_idx = chunk.index[-1] + 1  # Continue from the end of the current chunk

    return group

Draught calculations: 

In [15]:
def modify_edgelist_with_draught_changes(edge_df):
    # Ensure the Time column is in datetime format for proper handling
    edge_df['Start time'] = pd.to_datetime(edge_df['Start time'])
    
    # Sort the DataFrame by Time to ensure grouping works correctly
    edge_df = edge_df.sort_values('Start time')
    
    # # Create start and end draught columns for Ship1
    # # edge_df['start_draught1'] = edge_df.groupby(['Ship1', 'Ship2'])['initial_draught_1'].transform('first')
    # # edge_df['end_draught1'] = edge_df.groupby(['Ship1', 'Ship2'])['final_draught_1'].transform('last')
    
    # # Create start and end draught columns for Ship2
    # edge_df['start_draught2'] = edge_df.groupby(['Ship1', 'Ship2'])['initial_draught_2'].transform('first')
    # edge_df['end_draught2'] = edge_df.groupby(['Ship1', 'Ship2'])['final_draught_2'].transform('last')

    # Calculate absolute draught change for each ship in the pair
    edge_df['absolute_draught_change1'] = edge_df['final_draught_1'] - edge_df['initial_draught_1'] 
    edge_df['absolute_draught_change2'] = edge_df['final_draught_2'] - edge_df['initial_draught_2'] 

    return edge_df

Archimedes function for calculating weight: 

In [16]:
def archimedes(df):
    """
    Calculate weight change using Archimedes' principle for both Ship1 and Ship2.

    Parameters:
        df (pd.DataFrame): DataFrame containing sides_raw1, sides_raw2, 
                           absolute_draught_change1, and absolute_draught_change2.

    Returns:
        pd.DataFrame: Original DataFrame with new columns for weight change for Ship1 and Ship2.
    """
    # Ensure the required columns exist in the DataFrame
    required_columns = ['mean_A1', 'mean_B1', 'mean_C1', 'mean_D1', 'mean_A2', 'mean_B2', 'mean_C2', 'mean_D2', 'absolute_draught_change1', 'absolute_draught_change2']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")

    # Calculate weight change for Ship1 and Ship2
    df['weight_change_ship1(metric tons)'] = (0.81 * ((df['mean_A1'] + df['mean_B1']) * (df['mean_C1'] + df['mean_D1'])) * df['absolute_draught_change1'] * 1025)/1000
    df['weight_change_ship2(metric tons)'] = (0.81 * ((df['mean_A2'] + df['mean_B2']) * (df['mean_C2'] + df['mean_D2'])) * df['absolute_draught_change2'] * 1025)/1000

    return df

Function for determining weight directionality:

In [17]:
def determine_directionality(row):
    if row['absolute_draught_change1'] > 0 and row['absolute_draught_change2'] < 0:
        return 'Ship1 <- Ship2'
    elif row['absolute_draught_change1'] < 0 and row['absolute_draught_change2'] > 0:
        return 'Ship1 -> Ship2'
    elif row['absolute_draught_change1'] == 0 and row['absolute_draught_change2'] < 0:
        return 'Ship1(0) Ship2(-)'
    elif row['absolute_draught_change1'] == 0 and row['absolute_draught_change2'] > 0:
        return 'Ship1(0) Ship2(+)'
    elif row['absolute_draught_change1'] > 0 and row['absolute_draught_change2'] == 0:
        return 'Ship1(+) Ship2(0)'
    elif row['absolute_draught_change1'] < 0 and row['absolute_draught_change2'] == 0:
        return 'Ship1(-) Ship2(0)'
    elif row['absolute_draught_change1'] == 0 and row['absolute_draught_change2'] == 0:
        return 'None'
    elif row['absolute_draught_change1'] < 0 and row['absolute_draught_change2'] < 0:
        return 'Ship1(-) Ship2(-)'
    elif row['absolute_draught_change1'] > 0 and row['absolute_draught_change2'] > 0:
        return 'Ship1(+) Ship2(+)'

## Running the functions for 200 meters as a threshold

For 200 meters consecutive (adjust as necessary for 100 meters):

In [10]:
# creating edge_df_200 
start_time = time.time()
edge_df_200 = create_edgelist(df, 200) # adjust distance as necessary
edge_time = time.time() - start_time
print(f"How long it took to compute: {edge_time:.4f} seconds")

C:\Users\zhout\AppData\Local\Temp\ipykernel_2108\1096783826.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edge_df_cleaned["lat_long_date_hour"] = (


How long it took to compute: 1430.1343 seconds


In [11]:
edge_df_200.to_csv('our_algorithm3.csv', index=False)

In [18]:
# Navstat pre-processing. Assigning NaN values to navstat = 15
edge_df_200['NAVSTAT1'] = edge_df_200['NAVSTAT1'].replace(15, np.nan)
edge_df_200['NAVSTAT2'] = edge_df_200['NAVSTAT2'].replace(15, np.nan)
edge_df_200['A1'] = edge_df_200['A1'].replace(0, np.nan)
edge_df_200['B1'] = edge_df_200['B1'].replace(0, np.nan)
edge_df_200['C1'] = edge_df_200['C1'].replace(0, np.nan)
edge_df_200['D1'] = edge_df_200['D1'].replace(0, np.nan)
edge_df_200['A2'] = edge_df_200['A2'].replace(0, np.nan)
edge_df_200['B2'] = edge_df_200['B2'].replace(0, np.nan)
edge_df_200['C2'] = edge_df_200['C2'].replace(0, np.nan)
edge_df_200['D2'] = edge_df_200['D2'].replace(0, np.nan)

In [19]:
# Ensure 'Time' column is in datetime format
edge_df_200['Time'] = pd.to_datetime(edge_df_200['Time'])

# Group by Ship1 and Ship2, and apply the mean function
result_200 = edge_df_200.groupby(['Ship1', 'Ship2'], group_keys=False).apply(mean_values)

# Reset index if needed
result_200.reset_index(drop=True, inplace=True)

C:\Users\zhout\anaconda3\envs\Maritime\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\zhout\anaconda3\envs\Maritime\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\zhout\anaconda3\envs\Maritime\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\zhout\anaconda3\envs\Maritime\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\zhout\AppData\Local\Temp\ipykernel_2108\686991853.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from 

In [20]:
# DROP NAVSTAT1,2, ABCD 1,2, SPEED1,2
result_200 = result_200.drop(['SPEED1', 'SPEED2', 'NAVSTAT1', 'NAVSTAT2', 'A1', 'B1', 'C1', 'D1', 
                      'A2', 'B2','C2','D2', 'sorted_pair'], axis=1)

In [38]:
start_time = time.time()
edgelist_3_200 = find_consecutive_adjacency(result_200, 3, 3) # at least 3 consequtive hours
edge_time = time.time() - start_time
print(f"How long it took to compute: {edge_time:.4f} seconds")

How long it took to compute: 5.4738 seconds


In [39]:
edgelist_3_200.to_csv('3_200_3t2.csv', index=False)

In [60]:
pd.set_option('display.max_columns', None)
edgelist_3_200

,Ship1,Ship2,Start time,End time,Consecutive (hours),mean_speed,initial_flag1,final_flag1,initial_flag2,final_flag2,initial_mmsi1,final_mmsi1,initial_mmsi2,final_mmsi2,initial_owner1,final_owner1,initial_owner2,final_owner2,initial_manager1,final_manager1,initial_manager2,final_manager2,initial_draught1,final_draught1,initial_draught2,final_draught2,initial_latitude1,initial_longitude1,final_latitude1,final_longitude1,initial_latitude2,initial_longitude2,final_latitude2,final_longitude2,initial_course1,final_course1,initial_course2,final_course2,TYPENAME1,TYPENAME2,BUILT1,BUILT2,GT1,GT2,DWT1,DWT2,LOA1,LOA2,BEAM1,BEAM2,CLASS2,CLASS2,CLASSCODE1,CLASSCODE2,BUILDER1,BUILDER2,mean_A1,mean_B1,mean_C1,mean_D1,mean_A2,mean_B2,mean_C2,mean_D2,median_navstat1,median_navstat2,initial_heading1,final_heading1,initial_heading2,final_heading2,initial_name1,final_name1,initial_name2,final_name2,initial_destination1,final_destination1,initial_destination2,final_destination2,initial_eta1,final_eta1,initial_eta2,final_eta2
0,9301732,9256860,2022-05-23 16:00:00,2022-06-01 11:00:00,212.0,0.008491,Liberia (Republic of),Liberia (Republic of),Russian Federation,Russian Federation,555555555,636012628,273218760,999999999,BAXTER TRADING INC,BAXTER TRADING INC,MG-FLOT LLC,MG-FLOT LLC,DYNACOM TANKERS MANAGEMENT LTD,DYNACOM TANKERS MANAGEMENT LTD,MG-FLOT LLC,MG-FLOT LLC,7.5,11.9,14.7,14.7,37.94472,24.44290,37.94818,24.43956,37.94496,24.44326,37.94990,24.44003,40.8,185.5,17.8,17.8,Crude Oil Tanker,Crude Oil Tanker,2006,2003,38832,61991,70377,115527,229.0,249.0,32.2,44.0,AMERICAN BUREAU OF SHIPPING,RUSSIAN MARITIME REGISTER OF SHIPPING,ABS,RS,ONOMICHI DOCKYARD,SANOYAS MIZUSHIMA WORKS & SHIPYARD,189.792453,38.207547,25.0,7.0,207.735849,41.264151,18.0,26.0,5.0,1.0,16,200,511,511,ICE ENERGY,ICE ENERGY,LANA,LANA,KARYSTOS,PIRAEUS,FOR_ORDER,FOR_ORDER,06-15 15:00,06-15 15:00,04-14 12:00,04-14 12:00
1,9301732,9256860,2022-08-11 15:00:00,2022-08-14 15:00:00,73.0,0.008491,Liberia (Republic of),Liberia (Republic of),Iran (Islamic Republic of),Iran (Islamic Republic of),111111111,636012628,222222222,422544900,BAXTER TRADING INC,BAXTER TRADING INC,MG-FLOT LLC,MG-FLOT LLC,DYNACOM TANKERS MANAGEMENT LTD,DYNACOM TANKERS MANAGEMENT LTD,MG-FLOT LLC,MG-FLOT LLC,11.9,7.5,10.0,10.0,37.87018,23.52436,37.87304,23.52639,37.87005,23.52476,37.87313,23.52682,220.8,282.2,355.2,355.2,Crude Oil Tanker,Crude Oil Tanker,2006,2003,38832,61991,70377,115527,229.0,249.0,32.2,44.0,AMERICAN BUREAU OF SHIPPING,RUSSIAN MARITIME REGISTER OF SHIPPING,ABS,RS,ONOMICHI DOCKYARD,SANOYAS MIZUSHIMA WORKS & SHIPYARD,189.792453,38.207547,25.0,7.0,207.735849,41.264151,18.0,26.0,5.0,1.0,83,164,511,511,ICE ENERGY,ICE ENERGY,LANA,LANA,PIRAEUS,FOR ORDERS,PIRAEUS,PIRAEUS,06-02 00:30,01-01 00:30,01-01 18:00,07-02 18:00


In [37]:
start_time = time.time()
edgelist_24_200 = find_consecutive_adjacency_broad(edgelist_3_200)
edge_time = time.time() - start_time
print(f"How long it took to compute: {edge_time:.4f} seconds")

How long it took to compute: 42.4070 seconds


In [38]:
# draught
edgelist_24_200 = modify_edgelist_with_draught_changes(edgelist_24_200)

In [39]:
# directionality
edgelist_24_200['Directionality'] = edgelist_24_200.apply(determine_directionality, axis=1)

In [96]:
edgelist_3_200.to_csv('edge_list_all_200_3_consecutive_v6_1.csv', index = False)
edgelist_24_200.to_csv('edge_list_all_200_3_24_consecutive_v6_1.csv', index = False)
# edgelist_24_200.to_csv('v6_24.csv', index = False)

## Incorporate draught & weight calculations

In [34]:
# weight
archimedes(edgelist_24_200)

,Ship1,Ship2,Start time,End time,Consecutive (hours),mean_speed,initial_latitude_1,initial_longitude_1,final_latitude_1,final_longitude_1,...,mean_B1,mean_C1,mean_D1,mean_A2,mean_B2,mean_C2,mean_D2,Directionality,weight_change_ship1(metric tons),weight_change_ship2(metric tons)
3068,9337157,9223887,2021-01-01 00:00:00,2021-01-02 00:00:00,24.0,0.058333,2.34089,101.95010,2.34060,101.95023,...,52.0,23.0,37.0,279.0,55.0,26.000000,32.000000,None,0.0,0.0000
7925,9519717,9233208,2021-01-01 00:00:00,2021-01-07 07:00:00,151.0,0.026552,26.18810,50.67236,26.18805,50.67241,...,56.0,26.0,34.0,211.0,38.0,NaN,NaN,None,0.0,NaN
8851,9573660,9232450,2021-01-01 00:00:00,2021-01-02 19:00:00,43.0,0.001136,37.20810,15.19268,37.20802,15.19288,...,33.0,22.0,10.0,205.0,39.0,27.000000,15.000000,None,0.0,0.0000
1333,9289178,9270490,2021-01-01 00:00:00,2021-01-01 13:00:00,13.0,0.003571,18.12392,-94.40518,18.12388,-94.40520,...,107.0,16.0,16.0,190.0,38.0,12.000000,20.000000,None,0.0,0.0000
9210,9579561,9417000,2021-01-01 00:00:00,2021-01-02 19:00:00,43.0,0.029545,57.50815,10.69883,57.50806,10.69860,...,42.0,31.0,17.0,209.0,35.0,26.000000,16.000000,Ship1(0) Ship2(-),0.0,-17016.8040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10653,9612911,9328170,2023-12-31 16:00:00,2023-12-31 23:00:00,7.0,0.039286,43.90789,28.97885,43.90280,28.97801,...,NaN,NaN,17.0,213.0,37.0,25.142857,18.857143,None,NaN,0.0000
3735,9365776,9246279,2023-12-31 19:00:00,2023-12-31 23:00:00,4.0,0.391667,2.13742,104.92771,2.13620,104.92800,...,54.0,33.0,27.0,281.0,52.0,25.000000,35.000000,None,0.0,0.0000
278,9239848,9237620,2023-12-31 19:00:00,2023-12-31 23:00:00,4.0,0.084848,1.24401,103.56379,1.24777,103.56284,...,42.0,26.0,22.0,285.0,48.0,30.000000,30.000000,None,0.0,0.0000
12964,9733959,9534004,2023-12-31 20:00:00,2023-12-31 23:00:00,3.0,0.050000,27.82092,-97.20671,27.82094,-97.20660,...,54.0,23.0,37.0,281.0,52.0,29.000000,31.000000,Ship1(0) Ship2(+),0.0,4976.5185


## View and save new dataframe as CSV

In [57]:
pd.set_option('display.max_columns', None)
edgelist_24_200.head(3)

,Ship1,Ship2,Start time,End time,Consecutive (hours),mean_speed,initial_latitude_1,initial_longitude_1,final_latitude_1,final_longitude_1,initial_latitude_2,initial_longitude_2,final_latitude_2,final_longitude_2,initial_flag1,final_flag1,initial_flag2,final_flag2,initial_owner1,final_owner1,initial_owner2,final_owner2,initial_manager1,final_manager1,initial_manager2,final_manager2,MMSI1,MMSI2,initial_draught_1,final_draught_1,initial_draught_2,final_draught_2,absolute_draught_change1,absolute_draught_change2,mean_A1,mean_B1,mean_C1,mean_D1,mean_A2,mean_B2,mean_C2,mean_D2,Directionality
0,8507248,8228646,2022-03-16 14:00:00,2022-03-22 08:00:00,138.0,NaN,40.36045,49.9313,40.36053,49.93128,40.36076,49.93321,40.36083,49.93313,Azerbaijani Republic,Azerbaijani Republic,Azerbaijani Republic,Azerbaijani Republic,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,MERIDIAN SHIPPING & MANAGEMENT,MERIDIAN SHIPPING & MANAGEMENT,423024100,423036100,3.4,3.4,3.4,3.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,8507248,8228646,2022-06-03 13:00:00,2022-06-03 22:00:00,9.0,NaN,40.36045,49.9313,40.36053,49.93128,40.36076,49.93321,40.36083,49.93313,Azerbaijani Republic,Azerbaijani Republic,Azerbaijani Republic,Azerbaijani Republic,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,MERIDIAN SHIPPING & MANAGEMENT,MERIDIAN SHIPPING & MANAGEMENT,423024100,423036100,3.9,3.9,4.0,4.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,8507248,8228646,2022-06-06 07:00:00,2022-06-08 13:00:00,54.0,NaN,40.36045,49.9313,40.36053,49.93128,40.36076,49.93321,40.36083,49.93313,Azerbaijani Republic,Azerbaijani Republic,Azerbaijani Republic,Azerbaijani Republic,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,AZERBAIJAN CASPIAN SHIPPING,MERIDIAN SHIPPING & MANAGEMENT,MERIDIAN SHIPPING & MANAGEMENT,423024100,423036100,3.0,3.0,3.4,3.4,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [56]:
edge_df_200.to_csv(r'C:/Users/golov/Documents/maritine_analysis_home_pc/vessel_finder_edge_lists/edge_list_all_200_raw_v6.csv', index = False)
edgelist_3_200.to_csv(r'C:/Users/golov/Documents/maritine_analysis_home_pc/vessel_finder_edge_lists/edge_list_all_200_3_consecutive_v6.csv', index = False)
edgelist_24_200.to_csv(r'C:/Users/golov/Documents/maritine_analysis_home_pc/vessel_finder_edge_lists/edge_list_all_200_3_24_consecutive_v6.csv', index = False)


In [57]:
## Running the functions for 100 meters as a threshold

In [58]:
# creating edge_df_100 
start_time = time.time()
edge_df_100 = create_edgelist(df, 100) # adjust distance as necessary
edge_time = time.time() - start_time
print(f"How long it took to compute: {edge_time:.4f} seconds")

C:\Users\golov\AppData\Local\Temp\ipykernel_8352\1096783826.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edge_df_cleaned["lat_long_date_hour"] = (


How long it took to compute: 11163.7059 seconds


In [98]:
edge_df_100  = pd.read_csv('edge_list_all_100_raw_v6.csv')

In [99]:
# Navstat pre-processing. Assigning NaN values to navstat = 15
edge_df_100['NAVSTAT1'] = edge_df_100['NAVSTAT1'].replace(15, np.nan)
edge_df_100['NAVSTAT2'] = edge_df_100['NAVSTAT2'].replace(15, np.nan)
edge_df_100['A1'] = edge_df_100['A1'].replace(0, np.nan)
edge_df_100['B1'] = edge_df_100['B1'].replace(0, np.nan)
edge_df_100['C1'] = edge_df_100['C1'].replace(0, np.nan)
edge_df_100['D1'] = edge_df_100['D1'].replace(0, np.nan)
edge_df_100['A2'] = edge_df_100['A2'].replace(0, np.nan)
edge_df_100['B2'] = edge_df_100['B2'].replace(0, np.nan)
edge_df_100['C2'] = edge_df_100['C2'].replace(0, np.nan)
edge_df_100['D2'] = edge_df_100['D2'].replace(0, np.nan)

In [100]:
# Ensure 'Time' column is in datetime format
edge_df_100['Time'] = pd.to_datetime(edge_df_100['Time'])

# Group by Ship1 and Ship2, and apply the mean function
result_100 = edge_df_100.groupby(['Ship1', 'Ship2'], group_keys=False).apply(mean_values)

# Reset index if needed
result_100.reset_index(drop=True, inplace=True)

In [101]:
# DROP NAVSTAT1,2, ABCD 1,2, SPEED1,2
result_100 = result_100.drop(['SPEED1', 'SPEED2', 'NAVSTAT1', 'NAVSTAT2', 'A1', 'B1', 'C1', 'D1', 
                      'A2', 'B2','C2','D2', 'sorted_pair'], axis=1)

In [102]:
start_time = time.time()
edgelist_3_100 = find_consecutive_adjacency(result_100, 3,2) # at least 3 consequtive hours
edge_time = time.time() - start_time
print(f"How long it took to compute: {edge_time:.4f} seconds")

How long it took to compute: 31.3227 seconds


In [103]:
start_time = time.time()
edgelist_24_100 = find_consecutive_adjacency_broad(edgelist_3_100)
edge_time = time.time() - start_time
print(f"How long it took to compute: {edge_time:.4f} seconds")

How long it took to compute: 52.5193 seconds


In [64]:
#checking if all the required columns were created
edgelist_24_100.columns

Index(['Ship1', 'Ship2', 'Start time', 'End time', 'Consecutive (hours)',
       'mean_speed', 'initial_latitude_1', 'initial_longitude_1',
       'final_latitude_1', 'final_longitude_1', 'initial_latitude_2',
       'initial_longitude_2', 'final_latitude_2', 'final_longitude_2',
       'initial_flag1', 'final_flag1', 'initial_flag2', 'final_flag2',
       'initial_owner1', 'final_owner1', 'initial_owner2', 'final_owner2',
       'initial_manager1', 'final_manager1', 'initial_manager2',
       'final_manager2', 'MMSI1', 'MMSI2', 'initial_draught_1',
       'final_draught_1', 'initial_draught_2', 'final_draught_2',
       'absolute_draught_change1', 'absolute_draught_change2', 'mean_A1',
       'mean_B1', 'mean_C1', 'mean_D1', 'mean_A2', 'mean_B2', 'mean_C2',
       'mean_D2'],
      dtype='object')

In [104]:
# draught
edgelist_24_100 = modify_edgelist_with_draught_changes(edgelist_24_100)

In [105]:
# directionality
edgelist_24_100['Directionality'] = edgelist_24_100.apply(determine_directionality, axis=1)

In [106]:
edgelist_3_100.to_csv('edge_list_all_100_3_consecutive_v6_1.csv', index = False)
edgelist_24_100.to_csv('edge_list_all_100_3_24_consecutive_v6_1.csv', index = False)

In [66]:
# weight
archimedes(edgelist_24_100)

,Ship1,Ship2,Start time,End time,Consecutive (hours),mean_speed,initial_latitude_1,initial_longitude_1,final_latitude_1,final_longitude_1,initial_latitude_2,initial_longitude_2,final_latitude_2,final_longitude_2,initial_flag1,final_flag1,initial_flag2,final_flag2,initial_owner1,final_owner1,initial_owner2,final_owner2,initial_manager1,final_manager1,initial_manager2,final_manager2,MMSI1,MMSI2,initial_draught_1,final_draught_1,initial_draught_2,final_draught_2,absolute_draught_change1,absolute_draught_change2,mean_A1,mean_B1,mean_C1,mean_D1,mean_A2,mean_B2,mean_C2,mean_D2,weight_change_ship1(metric tons),weight_change_ship2(metric tons)
1549,9304356,9183300,2021-01-01 00:00:00,2021-01-01 09:00:00,9.0,0.020000,25.24941,55.26454,25.24944,55.26461,25.24905,55.26427,25.24905,55.26430,None,None,None,None,TRIGLAV SHIPPING INC,TRIGLAV SHIPPING INC,KAZBEG SHIPPING INC,KAZBEG SHIPPING INC,TRIGLAV SHIPPING INC,TRIGLAV SHIPPING INC,TEODOR OCEANIC DMCC,TEODOR OCEANIC DMCC,577450000,577338000,5.8,5.8,8.5,8.5,0.0,0.0,210.0,31.0,23.0,19.0,203.0,41.0,17.0,25.0,0.0000,0.0
960,9289178,9270490,2021-01-01 00:00:00,2021-01-01 13:00:00,13.0,0.003571,18.12392,-94.40518,18.12388,-94.40520,18.12397,-94.40573,18.12396,-94.40576,Liberia (Republic of),Liberia (Republic of),Liberia (Republic of),Liberia (Republic of),VIVIAN MARINE INC,VIVIAN MARINE INC,CANNA INTERNATIONAL INC,CANNA INTERNATIONAL INC,PLEIADES SHIPPING AGENTS SA,PLEIADES SHIPPING AGENTS SA,CANNA INTERNATIONAL INC,CANNA INTERNATIONAL INC,636012425,636090747,7.0,7.0,8.0,8.0,0.0,0.0,106.0,107.0,16.0,16.0,190.0,38.0,12.0,20.0,0.0000,0.0
10379,9787182,9211999,2021-01-01 00:00:00,2021-01-01 10:00:00,10.0,0.022727,44.90709,36.79872,44.90709,36.79829,44.90708,36.79832,44.90710,36.79835,Greece,Greece,None,None,JIRO SHIPPING SA,JIRO SHIPPING SA,SUNOR ILLC,SUNOR ILLC,MINERVA MARINE INC,MINERVA MARINE INC,ROSEWOOD SHIPPING LLC,ROSEWOOD SHIPPING LLC,241604000,229483000,10.3,10.3,8.2,8.2,0.0,0.0,214.0,36.0,28.0,16.0,212.0,37.0,19.0,25.0,0.0000,0.0
1171,9292498,9088689,2021-01-01 00:00:00,2021-01-14 05:00:00,317.0,0.036278,25.25515,55.26962,25.25513,55.26955,25.25545,55.26993,25.25548,55.26993,Puerto Rico,Puerto Rico,Marshall Islands (Republic of the),Marshall Islands (Republic of the),GRAND FINANCING CO,GRAND FINANCING CO,DIGIFRO OIL & GAS SERVICES LTD,DIGIFRO OIL & GAS SERVICES LTD,SMART TANKERS INC,SMART TANKERS INC,DIGIFRO OIL & GAS SERVICES LTD,DIGIFRO OIL & GAS SERVICES LTD,355943000,538006511,10.5,10.5,10.8,10.8,0.0,0.0,281.0,52.0,34.0,24.0,280.0,52.0,28.0,30.0,0.0000,0.0
11269,9817107,9321691,2021-01-01 00:00:00,2021-01-01 14:00:00,14.0,0.000000,27.81842,-97.20944,27.81843,-97.20942,27.81807,-97.20875,27.81805,-97.20870,Greece,Greece,Greece,Greece,EURONAV NV,EURONAV NV,PARADISE SHIPPING & MARINE INC,PARADISE SHIPPING & MARINE INC,EURONAV NV,EURONAV NV,PARADISE SHIPPING & MARINE INC,PARADISE SHIPPING & MARINE INC,241593000,240556000,10.0,13.7,8.8,8.8,3.7,0.0,227.0,50.0,26.0,22.0,231.0,43.0,17.0,31.0,40844.3148,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,9239848,9237620,2023-12-31 19:00:00,2023-12-31 23:00:00,4.0,0.081538,1.24539,103.56252,1.24777,103.56284,1.24577,103.56225,1.24822,103.56297,Cayman Islands,Cayman Islands,Panama (Republic of),Panama (Republic of),NAT BERMUDA HOLDINGS LTD,NAT BERMUDA HOLDINGS LTD,ITG ENERGY SINGAPORE PTE LTD,ITG ENERGY SINGAPORE PTE LTD,SCANDIC AMERICAN SHIPPING LTD,SCANDIC AMERICAN SHIPPING LTD,ITG ENERGY SINGAPORE PTE LTD,ITG ENERGY SINGAPORE PTE LTD,319099500,352978203,14.8,14.8,19.9,19.9,0.0,0.0,232.0,42.0,26.0,22.0,285.0,48.0,30.0,30.0,0.0000,0.0
11044,9804459,9334727,2023-12-31 20:00:00,2023-12-31 21:00:00,1.0,0.400000,44.59360,38.01900,44.59361,38.01899,44.59362,38.01906,44.59360,38.01899,Marshall Islands (Republic of the),Marshall Islands (Republic of the),Croatia (Republic of),Croatia (Republic of),STAVENTO SHIPOWNERS

In [68]:
pd.set_option('display.max_columns', None)
edgelist_24_100.head(3)

,Ship1,Ship2,Start time,End time,Consecutive (hours),mean_speed,initial_latitude_1,initial_longitude_1,final_latitude_1,final_longitude_1,initial_latitude_2,initial_longitude_2,final_latitude_2,final_longitude_2,initial_flag1,final_flag1,initial_flag2,final_flag2,initial_owner1,final_owner1,initial_owner2,final_owner2,initial_manager1,final_manager1,initial_manager2,final_manager2,MMSI1,MMSI2,initial_draught_1,final_draught_1,initial_draught_2,final_draught_2,absolute_draught_change1,absolute_draught_change2,mean_A1,mean_B1,mean_C1,mean_D1,mean_A2,mean_B2,mean_C2,mean_D2,weight_change_ship1(metric tons),weight_change_ship2(metric tons),Directionality
1549,9304356,9183300,2021-01-01,2021-01-01 09:00:00,9.0,0.020000,25.24941,55.26454,25.24944,55.26461,25.24905,55.26427,25.24905,55.26430,None,None,None,None,TRIGLAV SHIPPING INC,TRIGLAV SHIPPING INC,KAZBEG SHIPPING INC,KAZBEG SHIPPING INC,TRIGLAV SHIPPING INC,TRIGLAV SHIPPING INC,TEODOR OCEANIC DMCC,TEODOR OCEANIC DMCC,577450000,577338000,5.8,5.8,8.5,8.5,0.0,0.0,210.0,31.0,23.0,19.0,203.0,41.0,17.0,25.0,0.0,0.0,None
960,9289178,9270490,2021-01-01,2021-01-01 13:00:00,13.0,0.003571,18.12392,-94.40518,18.12388,-94.40520,18.12397,-94.40573,18.12396,-94.40576,Liberia (Republic of),Liberia (Republic of),Liberia (Republic of),Liberia (Republic of),VIVIAN MARINE INC,VIVIAN MARINE INC,CANNA INTERNATIONAL INC,CANNA INTERNATIONAL INC,PLEIADES SHIPPING AGENTS SA,PLEIADES SHIPPING AGENTS SA,CANNA INTERNATIONAL INC,CANNA INTERNATIONAL INC,636012425,636090747,7.0,7.0,8.0,8.0,0.0,0.0,106.0,107.0,16.0,16.0,190.0,38.0,12.0,20.0,0.0,0.0,None
10379,9787182,9211999,2021-01-01,2021-01-01 10:00:00,10.0,0.022727,44.90709,36.79872,44.90709,36.79829,44.90708,36.79832,44.90710,36.79835,Greece,Greece,None,None,JIRO SHIPPING SA,JIRO SHIPPING SA,SUNOR ILLC,SUNOR ILLC,MINERVA MARINE INC,MINERVA MARINE INC,ROSEWOOD SHIPPING LLC,ROSEWOOD SHIPPING LLC,241604000,229483000,10.3,10.3,8.2,8.2,0.0,0.0,214.0,36.0,28.0,16.0,212.0,37.0,19.0,25.0,0.0,0.0,None


In [69]:
edge_df_100.to_csv(r'C:/Users/golov/Documents/maritine_analysis_home_pc/vessel_finder_edge_lists/edge_list_all_100_raw_v6.csv', index = False)
edgelist_3_100.to_csv(r'C:/Users/golov/Documents/maritine_analysis_home_pc/vessel_finder_edge_lists/edge_list_all_100_3_consecutive_v6.csv', index = False)
edgelist_24_100.to_csv(r'C:/Users/golov/Documents/maritine_analysis_home_pc/vessel_finder_edge_lists/edge_list_all_100_3_24_consecutive_v6.csv', index = False)
